In [1]:
import os
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.FNO import FNO1d
from methods.methodsDataset.FNODataset import FNODataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_FNO = {'layer_dims': None, 'FourierLayers_modes': None, 'device': device, 'seed': 123}

In [4]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000).reshape(-1, 1)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)

In [7]:
D_train, D_val, U_train, U_val = sklearn.model_selection.train_test_split(D_list, U_sols,test_size=0.2, random_state=params_FNO['seed'])

In [8]:
D_fno_train = torch.Tensor(D_train.repeat(params_solver['nx'], 1)).unsqueeze(-1)
D_fno_val = torch.Tensor(D_val.repeat(params_solver['nx'], 1)).unsqueeze(-1)

In [10]:
def get_dataset():
    return D_fno_train, D_fno_val, U_train, U_val

In [11]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [12]:
def define_model(trial, input_chanel):
    n_fourier_layers = trial.suggest_int("n_fourier_layers", 1, 5)
    lifting_layer = trial.suggest_int("lifting_layer", 32, 64)
    layers = [input_chanel, lifting_layer]
    fourier_modes = []
    for i in range(n_fourier_layers):
        out_features = trial.suggest_int("Fourier_units_l{}".format(i), 32, 64)
        fourier_mode = trial.suggest_int("Fourier_modes_l{}".format(i), 12, 16)
        layers += [out_features]
        fourier_modes.append(fourier_mode)
    
    params_FNO_trial = copy.deepcopy(params_FNO)
    params_FNO_trial['layers_dim'] = layers
    params_FNO_trial['FourierLayers_modes'] = fourier_modes
    return FNO1d(params={'solver':params_solver, 'method':params_FNO_trial})

In [23]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1)

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)

    D_train, D_val, U_train, U_val = get_dataset()
    print(D_train.shape, D_val.shape, U_train.shape, U_val.shape)
    trainDataset = FNODataset(x=D_train, y=U_train)
    valDataset = FNODataset(x=D_val, y=U_val)
    batch_size = trial.suggest_int("batch_size", 170, 175)
    
    n_epochs = 10000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam', 'batch_size': batch_size}
    
    torch.manual_seed(params_FNO['seed'])
    model.fit(hyperparameters=hyperparameters, 
                            D_train=D_fno_train, D_val=D_fno_val, U_train=U_train, U_val=U_val)

            
    val_loss = model.loss_dict['val']
    trial.report(min(val_loss), n_epochs)
    
    return min(val_loss)

In [24]:
sampler = TPESampler(seed=params_FNO['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-09 17:07:28,704] A new study created in memory with name: no-name-f6f6d726-11e2-4537-8118-95913710ba5c


In [25]:
study.optimize(objective, n_trials=3)

torch.Size([800, 101, 1]) torch.Size([200, 101, 1]) (800, 101) (200, 101)


[epoch: 10000 : 100%|█████████████████████| 10001/10001 [04:22<00:00, 38.09it/s]
[I 2023-08-09 17:11:52,513] Trial 0 finished with value: 3.346859123176671e-09 and parameters: {'n_fourier_layers': 4, 'lifting_layer': 41, 'Fourier_units_l0': 39, 'Fourier_modes_l0': 14, 'Fourier_units_l1': 55, 'Fourier_modes_l1': 14, 'Fourier_units_l2': 64, 'Fourier_modes_l2': 15, 'Fourier_units_l3': 47, 'Fourier_modes_l3': 13, 'lr': 0.00010703702960745551, 'batch_size': 174}. Best is trial 0 with value: 3.346859123176671e-09.


torch.Size([800, 101, 1]) torch.Size([200, 101, 1]) (800, 101) (200, 101)


[epoch: 10000 : 100%|█████████████████████| 10001/10001 [03:23<00:00, 49.10it/s]
[I 2023-08-09 17:15:16,202] Trial 1 finished with value: 2.4169155288245037e-09 and parameters: {'n_fourier_layers': 3, 'lifting_layer': 33, 'Fourier_units_l0': 45, 'Fourier_modes_l0': 15, 'Fourier_units_l1': 38, 'Fourier_modes_l1': 12, 'Fourier_units_l2': 49, 'Fourier_modes_l2': 14, 'lr': 0.0008002079756292469, 'batch_size': 175}. Best is trial 1 with value: 2.4169155288245037e-09.


torch.Size([800, 101, 1]) torch.Size([200, 101, 1]) (800, 101) (200, 101)


[epoch: 10000 : 100%|█████████████████████| 10001/10001 [04:03<00:00, 41.06it/s]
[I 2023-08-09 17:19:19,785] Trial 2 finished with value: 3.6502567635210426e-09 and parameters: {'n_fourier_layers': 4, 'lifting_layer': 52, 'Fourier_units_l0': 55, 'Fourier_modes_l0': 13, 'Fourier_units_l1': 43, 'Fourier_modes_l1': 13, 'Fourier_units_l2': 41, 'Fourier_modes_l2': 15, 'Fourier_units_l3': 35, 'Fourier_modes_l3': 14, 'lr': 0.00019614998984838697, 'batch_size': 172}. Best is trial 1 with value: 2.4169155288245037e-09.


In [26]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  2.4169155288245037e-09
  Params: 
    n_fourier_layers: 3
    lifting_layer: 33
    Fourier_units_l0: 45
    Fourier_modes_l0: 15
    Fourier_units_l1: 38
    Fourier_modes_l1: 12
    Fourier_units_l2: 49
    Fourier_modes_l2: 14
    lr: 0.0008002079756292469
    batch_size: 175


In [27]:
modes = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'Fourier':
        if key.split('_')[1] == 'modes':
            modes.append(value)
        if key.split('_')[1] == 'units':
            layer_dims.append(value)
    elif key.split('_')[0] == 'lifting':
        lifting = value
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'batch_size':
        batch_size = value

Use this to fill config_step_1.py

In [29]:
print(layer_dims, '\n', modes, '\n', lifting, '\n', '\n', lr, '\n', batch_size)

[45, 38, 49] 
 [15, 12, 14] 
 33 
 
 0.0008002079756292469 
 175
